# Explanation Retrieval via Unification

## Unificationist account of explanation

One of the main function of an explanatory argument is to fit the explanandum (i.e., the event to be explained) into a broader pattern that maximises unification, showing that a set of apparently unrelated phenomena are part of a common regularity (Kitcher, 1981; Kitcher, 1989). From a linguistic point of view, the process of unification tends to generate sentence-level explanatory patterns that can be reused and instantiated for deriving and explaining many phenomena. In natural language, unification generally emerges as a process of abstraction from the explanandum through the implicit search of common high-level features and similarities between different phenomena.

<div>
    <img src="figures/unification.png", height="300">
    <img src="figures/unification_abstraction.png", height="300">
    <img src="figures/unification_patterns.png", height="300">
</div>

## Leveraging explanatory unification patterns for explanation retrieval

How do we model the unification power for explanation retrieval?

Given a sentence encoder $e(\cdot)$, we can model the explanatory relevance of a fact $f_i$ for a given hypothesis $h$ as follows:

- Standard Relevance:

\begin{equation}
\small
r(f_t, h_t) = sim(e(f_t), e(h_t))
\end{equation}

- Unification Power:

\begin{equation}
\small
pw(f_i, h) = \sum_{h_k \in kNN(h)}^K {sim(e(h),e(h_k))} \cdot 1_{E_k}(f_i)
\end{equation}  


\begin{equation}
\small
    1_{E_k}(f_i) =
        \begin{cases}
            1 & \text{if } f_i \in E_{k}\\
            0 & \text{if } f_i \notin E_{k}
        \end{cases}
\end{equation}

- Joint Explanatory Relevance:

\begin{equation}
\small
es(f_t, h_t) = \lambda \cdot r(f_t, h_t) + (1-\lambda) \cdot pw(f_t, h)
\end{equation}

## Explanation Retrieval in WorldTree

<div>
<img src="figures/worldtree_corpus.png" width="500">
<br><br>Distribution of explanatory sentences:<br><br>
<img src="figures/distribution_premises_worldtree.png" width="500">
</div>

Load the relevant libraries

In [1]:
from retrieval.retrieval_model import *
from evaluation.evaluation_model import ExplanationRetrievalEvaluation
from sskb import WorldTreeKB
from tqdm import tqdm
import numpy as np

c:\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Questions pre-processing. Convert into a hypothesis.

In [2]:
def preprocess_wt_questions(question):
    edit_surface = question.surface.split("(A)")[0]
    correct_answer = question.surface.split("("+question.annotations["AnswerKey"]+")")[1].split("(")[0]
    question.surface = edit_surface + " " + correct_answer
    return question

Load the WorldTree corpus

In [3]:
kb = WorldTreeKB()

train_questions = [stt for stt in kb if (stt.annotations["type"] == "question" and stt.annotations["split"] == "train")]
for question in train_questions:
    question = preprocess_wt_questions(question)

dev_questions = [stt for stt in kb if (stt.annotations["type"] == "question" and stt.annotations["split"] == "dev")]
for question in dev_questions:
    question = preprocess_wt_questions(question)

facts_kb = [stt for stt in kb if (stt.annotations["type"] == "fact")]

Loading fact tables: 100%|██████████| 82/82 [00:02<00:00, 37.54it/s] 
Loading questions [train]: 2207it [01:03, 34.52it/s]
Loading questions [dev]: 496it [00:13, 36.38it/s]
Loading questions [test]: 1664it [00:00, 2893.35it/s]


Print an example of hypothesis and explanation

In [4]:
print("\nRandom hypothesis from the Explanation KB:\n")

print("Hypothesis:\n")
print(dev_questions[42].surface)
print("\nExplanation:\n")
print("\n".join([premise.surface for premise in dev_questions[42].premises]))

print("\nRandom statements from the Facts KB:\n")
print(facts_kb[50].surface)
print(facts_kb[120].surface)


Random hypothesis from the Explanation KB:

Hypothesis:

Ocean tides of Earth are strongly influenced by the Moon. During which lunar phases are ocean tides lowest on Earth?   first quarter and last quarter

Explanation:

lunar phase is synonymous with moon phase
a first quarter is a kind of phase of the moon
a last quarter is a kind of phase of the moon
the gravitational pull of the Moon  on Earth's oceans causes the tides
as the gravitational pull of the moon on the Earth decreases , the size of the tides on Earth decrease
the lowest gravitational pull is during a quarter moon

Random statements from the Facts KB:

groundwater is located under ground; underground
nebulae are located outside the solar system


Initialize the explanation retrieval models

In [5]:
# Initialize BM25 model
bm25_retriever = BM25Model(facts_kb)
# Initialize Unification model
solved_cases = train_questions
unification_retriever = UnificationModel(facts_kb, solved_cases)
# Initialize Ensamble model
unification_weigth = 0.2
ensemble_model = EnsembleModel([bm25_retriever, unification_retriever], weights=[1-unification_weigth, unification_weigth])

Preprocessing - Unification: 100%|██████████| 2207/2207 [00:01<00:00, 1968.64it/s]


Perform explanation retrieval

In [6]:
num_examples = 50
    
queries_list = [hypothesis.surface for hypothesis in dev_questions[:num_examples]] 

# Compute BM25 and Unification scores
res_bm25 = bm25_retriever.query(queries_list)
res_unification = unification_retriever.query(queries_list, top_q = 20)
# Ensamble between BM25 and Unification model
res_ensemble = ensemble_model.query(queries_list, top_q = 20)

Unification - Processing queries: 100%|██████████| 50/50 [00:00<00:00, 82.65it/s]


Evaluation

In [7]:
references_id = "UID"

evaluation_model = ExplanationRetrievalEvaluation(dev_questions[:num_examples])

print("MAP BM5:", evaluation_model.evaluate(res_bm25, facts_kb)["map"])
print("MAP Unification:", evaluation_model.evaluate(res_unification, facts_kb)["map"])
print("MAP ensamble:", evaluation_model.evaluate(res_ensemble, facts_kb)["map"])

Processing evaluation: 100%|██████████| 50/50 [00:00<00:00, 342.18it/s]


MAP BM5: 0.22839637203072288


Processing evaluation: 100%|██████████| 50/50 [00:00<00:00, 365.17it/s]


MAP Unification: 0.3039692796796842


Processing evaluation: 100%|██████████| 50/50 [00:00<00:00, 347.41it/s]

MAP ensamble: 0.3871849125371243


Ranking Example

In [8]:
query_string = "What can explain why an apple fall to the ground and the Earth rotates around the Sun?"

bm25_res = bm25_retriever.query([query_string], top_k = 5)[0]

unification_res, similar_cases= unification_retriever.query([query_string], top_k = 5, top_q = 10, return_cases = True)
unification_res = unification_res[0]
similar_cases = similar_cases[0]

ensemble_res = ensemble_model.query([query_string], top_k = 5, top_q = 10)[0]

print("\nQuery:", query_string)
print("\n============== BM25 =================\n")
print("\n".join([facts_kb[int(res[0])].surface for res in bm25_res]))
print("\n=========== Unification ==============\n")
print("\n".join([facts_kb[int(res[0])].surface for res in unification_res]))

print("\nSimilar Cases:\n")
print("\n".join([train_questions[int(res[0])].surface for res in similar_cases]))

print("\n=========== Ensamble ==============\n")
print("\n".join([facts_kb[int(res[0])].surface for res in ensemble_res]))

Unification - Processing queries: 100%|██████████| 1/1 [00:00<00:00, 90.85it/s]


Query: What can explain why an apple fall to the ground and the Earth rotates around the Sun?

============== BM25 =================

the Earth revolves around the sun
hailing is when hail fall from clouds to the Earth;ground
snowing is when snow fall from clouds to the Earth;ground
the Earth rotates on its tilted axis
the Earth revolving around the Sun causes the seasons to change; to occur on its axis

=========== Unification ==============

the Sun is a kind of star
gravity;gravitational force causes objects that have mass; substances to be pulled down; to fall on a planet
a complete revolution; orbit of the Earth around the sun takes 1; one year; solar year; Earth year
a ball is a kind of object
years are a kind of unit for measuring time

Similar Cases:

Which of the following best explains why the Sun appears to move across the sky every day?   Earth rotates on its axis. 
A student drops a ball. Which force causes the ball to fall to the ground?   gravity 
Approximately how many

# Natural Language Premise Selection - ProofWiki

<div>
<img src="figures/premise_selection_proofwiki.png" width="700">
<br><br>Distribution of premises:<br><br>
<img src="figures/distribution_premises_proofwiki.png" width="400">
</div>

Load the ProofWiki corpus

In [2]:
from sskb import ProofWikiKB

kb = ProofWikiKB.from_resource("proofwiki[entities]")

train_propositions = [stt for stt in kb 
                        if (stt.annotations["type"] == "proposition" and stt.annotations["split"] == "train")]
dev_propositions = [stt for stt in kb 
                        if (stt.annotations["type"] == "proposition" and stt.annotations["split"] == "dev")]

facts_kb = [stt for stt in kb if (stt.annotations["type"] == "fact")]

Print an example of proposition and premises

In [9]:
print("Proposition:\n")
print(dev_propositions[35].surface)
print("\nPremises:\n")
print("\n\n".join([premise.surface for premise in dev_propositions[35].premises]))

Proposition:

Let $x \in \Z$ be an even integer.
Then $x + 5$ is odd.

Premises:

An integer $n \in \Z$ is '''even''' {{iff}} it is divisible by $2$.
An integer $n \in \Z$ is '''even''' {{iff}} it is of the form:
:$n = 2 r$
where $r \in \Z$ is an integer.
An integer $n \in \Z$ is '''even''' {{iff}}:
:$x \equiv 0 \pmod 2$
where the notation denotes congruence modulo $2$.
The set of even integers can be denoted $2 \Z$.
The first few non-negative even integers are:
:$0, 2, 4, 6, 8, 10, \ldots$
{{EuclidSaid}}
:''An '''even number''' is that which is divisible into two equal parts.''
{{EuclidDefRefNocat|VII|6|Even Number}}
Euclid went further and categorised even numbers according to:
: whether they are multiples of $4$
and:
: whether they have an odd divisor:
Let $n$ be an integer.
Then $n$ is '''even-times even''' {{iff}} it has $4$ as a divisor.
The first few non-negative '''even-times even''' numbers are:
:$0, 4, 8, 12, 16, 20, \ldots$
{{EuclidSaid}}
:''An '''even-times even number''' i

Proposition:

Let $x \in \Z$ be an even integer.
Then $x + 5$ is odd.

Premises:

An integer $n \in \Z$ is even if and only if it is divisible by $2$.
An integer $n \in \Z$ is even if and only if it is of the form:

$n = 2 r$

where $r \in \Z$ is an integer.

An integer $n \in \Z$ is even if and only if:

$x \equiv 0 \pmod 2$

where the notation denotes congruence modulo $2$.

The set of even integers can be denoted $2 \Z$.

The first few non-negative even integers are:

$0, 2, 4, 6, 8, 10, \ldots$

...


The numbers $\set {\ldots, -3, -2, -1, 0, 1, 2, 3, \ldots}$ are called the '''integers'''. This set is usually denoted $\Z$.

An individual element of $\Z$ is called '''an integer'''

...

Initialize the premise retrieval models

In [12]:
# Initialize BM25 model
bm25_retriever = BM25Model(facts_kb)
# Initialize Unification model
unification_retriever = UnificationModel(facts_kb, train_propositions)

Preprocessing - Unification: 100%|██████████| 5540/5540 [00:01<00:00, 3511.31it/s]


Perform premise retrieval

In [13]:
num_examples = 50
unification_weigth = 0.5

queries_list = [hypothesis.surface for hypothesis in dev_propositions[:num_examples]] 

# Compute BM25 and Unification scores
res_bm25 = bm25_retriever.query(queries_list)
res_unification = unification_retriever.query(queries_list, top_q = 10)

# Ensamble between BM25 and Unification model
res_ensamble = np.sum([(1-unification_weigth) * np.array(res_bm25), 
                        unification_weigth * np.array(res_unification)], axis = 0)

Unification - Processing queries: 100%|██████████| 50/50 [00:06<00:00,  7.65it/s]


Evaluation

In [14]:
references_id = "id"

evaluation_model = ExplanationRetrievalEvaluation(dev_propositions[:num_examples])

print("MAP BM5:", evaluation_model.evaluate(res_bm25, facts_kb)["map"])
print("MAP Unification:", evaluation_model.evaluate(res_unification, facts_kb)["map"])
print("MAP ensamble:", evaluation_model.evaluate(res_ensamble, facts_kb)["map"])

Processing evaluation: 100%|██████████| 50/50 [00:00<00:00, 192.72it/s]


MAP BM5: 0.10175165765410338


Processing evaluation: 100%|██████████| 50/50 [00:00<00:00, 247.34it/s]


MAP Unification: 0.24451411170722584


Processing evaluation: 100%|██████████| 50/50 [00:00<00:00, 168.14it/s]

MAP ensamble: 0.27093509360146223
